### Edward

I am going to try to translate chapter 2 examples to Edward.

AB testing

In [2]:
from scipy import stats
import numpy as np

#set constants
p_true = 0.05  # remember, this is unknown.
N = 1500

# sample N Bernoulli random variables from Ber(0.05).
# each random variable has a 0.05 chance of being a 1.
# this is the data-generation step
occurrences = stats.bernoulli.rvs(p_true, size=N)

print(occurrences) # Remember: Python treats True == 1, and False == 0
print(np.sum(occurrences))

[0 0 0 ..., 0 0 0]
65


In [ ]:
#pymc3
#include the observations, which are Bernoulli

import pymc3 as pm

# The parameters are the bounds of the Uniform.
with pm.Model() as model:
    p   = pm.Uniform('p', lower=0, upper=1)
    obs = pm.Bernoulli("obs", p, observed=occurrences)
    # To be explained in chapter 3
    step = pm.Metropolis()
    trace = pm.sample(18000, step=step)
    burned_trace = trace[1000:]

In [8]:

from edward.models import Bernoulli, Uniform, Normal, Empirical
from edward.inferences import MetropolisHastings
import tensorflow as tf
import numpy as np

p = Uniform(low=0., high=1.) 
x = Bernoulli(probs=p, sample_shape=1500) #sample shape matched to data array length

#not so sure about these...
mu = Normal(loc=0.0, scale=1.0)
#proposal_mu = Normal(loc=mu, scale=0.5)
q = Empirical(tf.Variable(tf.zeros(1500))) 

#try a different inference algo that I can understand?
#easier to try this? http://edwardlib.org/api/ed/HMC
#compare to pymc metropolis
# http://docs.pymc.io/api/inference.html?#module-pymc3.step_methods.metropolis

#for each random variable you need another variable that describes what you "think" it's distribution is. 
#not sure what my proposal vars should be...
inference = MetropolisHastings({p: q}, {}, data={x: occurrences})

In [ ]:
#edward - sample code

from edward.models import Beta, Bernoulli, Empirical
from edward import Gibbs
import tensorflow as tf
import numpy as np

x_data = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

p = Beta(1.0, 1.0)
x = Bernoulli(probs=p, sample_shape=10)

qp = Empirical(tf.Variable(tf.zeros(500)))
inference = Gibbs({p: qp}, data={x: x_data})

inference.run(n_iter=250) #run inference (Gibbs sampling)
x.sample(10).eval() #get some samples from x. 